In [1]:
# Definindo variáveis iniciais
import estrutura_ud
import os

root_deprel = ["root"]
subordinate_deprel = ["acl:relcl", "ccomp", "advcl"]
max_examples = 20

path = {
    'bosque': "bosque-ud-2.6.conllu",
    'dhbb': "dhbb.conllu",
    "obras": "obras.conllu"
}

In [2]:
# Carregamento dos corpora (demora muito!)
bosque = estrutura_ud.Corpus(recursivo=True)
dhbb = estrutura_ud.Corpus(recursivo=True)
obras = estrutura_ud.Corpus(recursivo=True)
bosque.load(path["bosque"])
dhbb.load(path["dhbb"])
obras.load(path["obras"])

corpora = {
    'dhbb': dhbb,
    'obras': obras,
    'bosque': bosque
}

In [3]:
# Estatísticas dos corpora
stats_corpora = {}
for corpus in corpora:
    stats_corpora[corpus] = {
        "size": os.path.getsize(path[corpus])/1024, 
        "n_tokens": 0, 
        "n_sentences": 0,
    }
    
for corpus in corpora:
    for sentence in corpora[corpus].sentences.values():
        stats_corpora[corpus]['n_sentences'] += 1
        for token in sentence.tokens:
            if not '-' in token.id:
                stats_corpora[corpus]['n_tokens'] += 1
    
print(stats_corpora)

{'dhbb': {'size': 960964.294921875, 'n_tokens': 16037286, 'n_sentences': 480218}, 'obras': {'size': 480188.513671875, 'n_tokens': 7863261, 'n_sentences': 353662}, 'bosque': {'size': 14846.041015625, 'n_tokens': 227825, 'n_sentences': 9364}}


In [4]:
# Números de orações
total_deprel = {}
for corpus in corpora:
    total_deprel[corpus] = {
        "|".join(root_deprel): 0,
        "|".join(subordinate_deprel): 0
    }
    for sentence in corpora[corpus].sentences.values():
        for token in sentence.tokens:
            if token.deprel in root_deprel:
                total_deprel[corpus]["|".join(root_deprel)] += 1
            if token.deprel in subordinate_deprel:
                total_deprel[corpus]["|".join(subordinate_deprel)] += 1
print(total_deprel)

{'dhbb': {'root': 480218, 'acl:relcl|ccomp|advcl': 341133}, 'obras': {'root': 353662, 'acl:relcl|ccomp|advcl': 316297}, 'bosque': {'root': 9364, 'acl:relcl|ccomp|advcl': 6842}}


# Busca ingênua tanto em root quanto em subordinada

In [5]:
def find_clause_without_subject(sentences, deprel_list):
    clauses_without_subject = {}
    for sent_id, sentence in sentences.items():
        for t, token in enumerate(sentence.tokens):
            if token.deprel in deprel_list:
                has_subj = False
                for _token in sentence.tokens:
                    if _token.dephead == token.id and _token.deprel in ["nsubj", "csubj", "nsubj:pass"]:
                        has_subj = True
                        break
                if not has_subj:
                    if not sent_id in clauses_without_subject:
                        clauses_without_subject[sent_id] = []
                    clauses_without_subject[sent_id].append(t)
    return clauses_without_subject

root_without_subject = {}
subordinate_without_subject = {}
for corpus in corpora:
    root_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, root_deprel)
    subordinate_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, subordinate_deprel)

In [6]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


dhbb - orações principais com sujeito oculto ingênuo: 226122 / 480218
dhbb - orações subordinadas com sujeito oculto ingênuo: 190381 / 341133

obras - orações principais com sujeito oculto ingênuo: 172124 / 353662
obras - orações subordinadas com sujeito oculto ingênuo: 156040 / 316297

bosque - orações principais com sujeito oculto ingênuo: 2777 / 9364
bosque - orações subordinadas com sujeito oculto ingênuo: 2617 / 6842

=== 20 exemplos de sujeito oculto na oração principal em dhbb ===
2: *Estudou* em o Ginásio Diocesano de São Paulo e bacharelou se em 1910 por a Faculdade de Ciências Jurídicas e Sociais .
3: Dedicando se a a advocacia , foi *promotor* público em Cunha ( SP ) e depois delegado de polícia em o Rio de Janeiro , então Distrito Federal .
4: *Iniciou* sua vida política como deputado federal por o Distrito Federal , exercendo o mandato de 1927 a 1929 .
6: Ligado a o governo federal , *encontrava* se a o lado de o presidente Washington Luís , em o palácio Guanabara , em o 

# Filtro A

In [7]:
def find_haver_impessoal(sentences, list_tokens):
    haver_impessoal = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma == "haver" and "Number=Sing" in token.feats and "Person=3" in token.feats:
                if not sent_id in haver_impessoal:
                    haver_impessoal[sent_id] = []
                haver_impessoal[sent_id].append(t)
    return haver_impessoal
                
root_haver_impessoal = {}
subordinate_haver_impessoal = {}
for corpus in corpora:
    root_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_haver_impessoal[corpus]:
        for token in root_haver_impessoal[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_haver_impessoal[corpus]:
        for token in subordinate_haver_impessoal[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [8]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in root_haver_impessoal[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in subordinate_haver_impessoal[corpus].values()])))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de haver impessoal na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_haver_impessoal[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de haver impessoal na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_haver_impessoal[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


dhbb - orações principais com sujeito oculto e haver impessoal: 1083
dhbb - orações subordinadas com sujeito oculto e haver impessoal: 861

obras - orações principais com sujeito oculto e haver impessoal: 5181
obras - orações subordinadas com sujeito oculto e haver impessoal: 4395

bosque - orações principais com sujeito oculto e haver impessoal: 124
bosque - orações subordinadas com sujeito oculto e haver impessoal: 148

=== 20 exemplos de haver impessoal na oração principal em dhbb ===
486: *Há* entretanto quem julgue , como Alexandre Barbosa Lima Sobrinho , que suas realizações foram motivadas por o afã de projetar nacionalmente o próprio nome de Antônio Carlos , tendo em vista a sucessão de Washington Luís , que assumira a presidência de a República em 15 de novembro de 1926 .
553: *Houve* idas e vindas a propósito de a data em que seria possível deflagrar a luta armada .
639: *Houve* críticas , formuladas , segundo Helena Bomeny , « a partir de o pressuposto de que o pacto entre 

232: Em a suavidade de as suas faces de moço , *havia* a sombra de as duras penitências e de as grandes vigílias místicas sobre as páginas de o breviário , ou defronte de o altar de a Virgem Santíssima , mas havia também uma juvenil frescura de flor , de essas misteriosas e pálidas , que só a a noite desabrocham e recendem .
251: Quando as suas primeiras palavras sairam lhe de o coração , em um doce murmúrio de voz angélica , *houve* em todas aquelas pobres criaturas , estafadas por o vício e por a libertinagem , uma inesperada comoção que lhes umedecia os olhos .
400: Não *há* dúvida que está aí , está cantando a primeira missa e entrando logo em seguida para a capela real .
408: Não *há* de ser com súplicas e sermões que a França se resgatará , mas a metralha , a canhão e a ponta de baionetas ! --
552: E em o arruído de as suas vozes ameaçadoras , *há* como que a repercussão de um inferno sufocado por as águas !
829: *Há* de convir que não dei o menor passo , nem empreguei o menor es

CP104-2: O deputado social-democrata Carlos Coelho , por exemplo , até já se deu a o trabalho de comparar os dois documentos , apenas para provar que *há* « inúmeras citações literais » de o diploma em o articulado de o pacto , que assim se transforma « em um repositório de ideias globais , que não acrescenta nada a o que a lei de bases e a Constituição portuguesa preconizam » .
CP113-4: Se não se incomodarem um a o outro é porque *há* condições para partir para a fase seguinte .
CP118-1: Mas em os demais casos em que a lei permite a cobrança coerciva por os tribunais fiscais , pode ser discutida a legalidade de a dívida , o que quer dizer que não *há* título executivo .
CP130-2: Para Jorge Rocha , o inquérito demonstrou « que *há* muitas câmaras de o país que têm uma atitude de grande dignidade por os animais » , apesar de serem letra morta as parcas disposições de a legislação que regulamenta esta matéria e de o imenso atraso em o nosso país em relação a outros países europeus , onde

# Filtro B

In [9]:
def find_nominals(sentences, list_tokens):
    nominals = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.upos != "VERB":
                is_cop = False
                for _token in sentence.tokens:
                    if _token.head_token.id == token.id and _token.deprel == "cop":
                        is_cop = True
                        break
                if not is_cop:
                    if not sent_id in nominals:
                        nominals[sent_id] = []
                    nominals[sent_id].append(t)
    return nominals
                
root_nominals = {}
subordinate_nominals = {}
for corpus in corpora:
    root_nominals[corpus] = find_nominals(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_nominals[corpus] = find_nominals(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_nominals[corpus]:
        for token in root_nominals[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_nominals[corpus]:
        for token in subordinate_nominals[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [10]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e nominal: {}".format(corpus, sum([len(x) for x in root_nominals[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e nominal: {}".format(corpus, sum([len(x) for x in subordinate_nominals[corpus].values()])))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de nominais como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_nominals[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de nominais como oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_nominals[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


dhbb - orações principais com sujeito oculto e nominal: 31599
dhbb - orações subordinadas com sujeito oculto e nominal: 735

obras - orações principais com sujeito oculto e nominal: 43326
obras - orações subordinadas com sujeito oculto e nominal: 2766

bosque - orações principais com sujeito oculto e nominal: 1145
bosque - orações subordinadas com sujeito oculto e nominal: 50

=== 20 exemplos de nominais como oração principal em dhbb ===
28: *Deputados* em fevereiro seguinte .
110: As *Oposições*
117: *Magalhães* , o voto de os representantes classistas e de os republicanos paulistas e ainda o beneplácito de Valadares .
120: *Transcorreria* em 1937 a campanha para a sucessão presidencial .
133: *Pinto* , visando promover uma manifestação de personalidades que contestavam o Estado Novo .
138: « *Manifesto* de os mineiros » , primeiro pronunciamento público de setores liberais contra o Estado Novo .
148: Aleixo fora *um* de os que propuseram o nome de Eduardo Gomes para as eleições pres

62: A *ferocidade* e a perfídia de os tempos bárbaros , os crimes de o feudalismo , todos os erros , todos os abusos e todos os desregramentos de um governo cínico e perverso e de uma magistratura e uma jurisprudência feitas de ignomínia e adulação , eis de o que se compunham os costumes de esse infeliz começo de século .
65: Só uma *coisa* divertia o público : -- a crápula .
82: Por mais *brilhante* e intensa que fosse a luz de o seu arrependimento , lá estava o gigantesco espectro de os crimes cometidos , para antepor se entre eles , e encher de sombra e remorso aquela consciência de sacerdote pecador .
83: Por mais *sincera* e convicta que fosse a sua nova lei de conduta , por mais leal e verdadeira a sua nova linha de virtude , sua alma chorava perdida para sempre , porque para sempre se sentia corrompida e suja .
87: *Francisco* de Paula , denominados -- « Os mínimos » .
98: *Deus* a os desamparados por os homens .
103: Eis a o que nem o próprio Ozéas seria capaz de responder *.*


CP206-4: A finalizar , o protesto solicita uma intervenção de o presidente de a Junta de Freguesia , « a fim de esclarecer esta situação ou , se *possível* , acabar com as sessões » .
CP342-7: Além de isso , a o proteger os samaritanos de Nablus , mostra que vai mais longe de o que o *Estado* hebraico , que não dá a os de Holon a oportunidade de estarem representados em o Knesset ( parlamento ) .
CP397-4: « Quanto maior o *contraste* detectado por os nossos olhos , mais rápido as coisas parecem andar » , diz o investigador .
CP442-4: E depois a juíza chamou o , vendo um homem com uma gabardina enorme , ainda maior que *ele* , um homem com cabelos grossos e brilhantes como crina de cavalo que tem andado a a chuva , sem acompanhamento veterinário e com pouca forragem .


# Filtro C

In [11]:
fenomenos_natureza = "chover|ventar|anoitecer|amanhecer|entardecer|relampejar|trovejar|escurecer|clarear".split("|")
def find_natureza(sentences, list_tokens):
    natureza = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma in fenomenos_natureza:
                if not sent_id in natureza:
                    natureza[sent_id] = []
                natureza[sent_id].append(t)
    return natureza
                
root_natureza = {}
subordinate_natureza = {}
for corpus in corpora:
    root_natureza[corpus] = find_natureza(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_natureza[corpus] = find_natureza(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_natureza[corpus]:
        for token in root_natureza[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_natureza[corpus]:
        for token in subordinate_natureza[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [12]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in root_natureza[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in subordinate_natureza[corpus].values()])))
    
for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de fenômeno natureza como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_natureza[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de fenômeno natureza como oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_natureza[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


dhbb - orações principais com sujeito oculto e fenômeno natureza: 0
dhbb - orações subordinadas com sujeito oculto e fenômeno natureza: 5

obras - orações principais com sujeito oculto e fenômeno natureza: 113
obras - orações subordinadas com sujeito oculto e fenômeno natureza: 147

bosque - orações principais com sujeito oculto e fenômeno natureza: 1
bosque - orações subordinadas com sujeito oculto e fenômeno natureza: 2

=== 20 exemplos de fenômeno natureza como oração principal em dhbb ===

=== 20 exemplos de fenômeno natureza como oração subordinada em dhbb ===
5692: Que o Lula , de botar tanto poste sem luz , acabe *escurecendo* o Brasil .
82280: Em Cachoeira de o Sul , o levante se desenrolou sem qualquer contratempo : em o *entardecer* de o dia 4 de outubro , as unidades federais de a cidade já se achavam reorganizadas por os revolucionários , prontas para seguir para o norte de o estado , de onde rumariam para São Paulo e Rio .
337330: Em o segundo dia de luta , uma companhia 

# Filtro D

In [13]:
def find_se(sentences, list_tokens):
    se = {"a": {}, "b": {}, "c": {}}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and "Gender=Unsp" in token.feats and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                if not sent_id in se["a"]:
                    se["a"][sent_id] = []
                se["a"][sent_id].append(sentence.map_token_id[token.head_token.id])
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and "VerbForm=Inf" in token.head_token.feats and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                if not sent_id in se["b"]:
                    se["b"][sent_id] = []
                se["b"][sent_id].append(sentence.map_token_id[token.head_token.id])
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                for _token in sentence.tokens:
                    if _token.deprel == "case" and _token.head_token.head_token.id == token.head_token.id:
                        if not sent_id in se["c"]:
                            se["c"][sent_id] = []
                        se["c"][sent_id].append(sentence.map_token_id[token.head_token.id])
                        break
    return se
                
root_se = {}
subordinate_se = {}
for corpus in corpora:
    root_se[corpus] = find_se(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_se[corpus] = find_se(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_se[corpus]["a"]:
        for token in root_se[corpus]["a"][sentence]:
            if token in root_without_subject[corpus][sentence]:
                root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_se[corpus]["a"]:
        for token in subordinate_se[corpus]["a"][sentence]:
            if token in subordinate_without_subject[corpus][sentence]:
                subordinate_without_subject[corpus][sentence].remove(token)
            
    for sentence in root_se[corpus]["b"]:
        for token in root_se[corpus]["b"][sentence]:
            if token in root_without_subject[corpus][sentence]:
                root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_se[corpus]["b"]:
        for token in subordinate_se[corpus]["b"][sentence]:
            if token in subordinate_without_subject[corpus][sentence]:
                subordinate_without_subject[corpus][sentence].remove(token)
            
    for sentence in root_se[corpus]["c"]:
        for token in root_se[corpus]["c"][sentence]:
            if token in root_without_subject[corpus][sentence]:
                root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_se[corpus]["c"]:
        for token in subordinate_se[corpus]["c"][sentence]:
            if token in subordinate_without_subject[corpus][sentence]:
                subordinate_without_subject[corpus][sentence].remove(token)

In [14]:
for corpus in corpora:
    for filtro in ["a", "b", "c"]:
        print("\n{} - orações principais com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in root_se[corpus][filtro].values()])))
        print("{} - orações subordinadas com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in subordinate_se[corpus][filtro].values()])))

for corpus in corpora:
    for filtro in ["a", "b", "c"]:
        i = 0
        ii = 0
        print("\n=== {} exemplos de se (filtro {}) como filho de oração principal em {} ===".format(max_examples, filtro, corpus))
        for sent_id, tokens in root_se[corpus][filtro].items():
            i += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if i >= max_examples:
                break
        print("\n=== {} exemplos de se (filtro {}) como filho de oração subordinada em {} ===".format(max_examples, filtro, corpus))
        for sent_id, tokens in subordinate_se[corpus][filtro].items():
            ii += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if ii >= max_examples:
                break


dhbb - orações principais com sujeito oculto e se (filtro a): 59
dhbb - orações subordinadas com sujeito oculto e se (filtro a): 757

dhbb - orações principais com sujeito oculto e se (filtro b): 79
dhbb - orações subordinadas com sujeito oculto e se (filtro b): 2692

dhbb - orações principais com sujeito oculto e se (filtro c): 26425
dhbb - orações subordinadas com sujeito oculto e se (filtro c): 13189

obras - orações principais com sujeito oculto e se (filtro a): 215
obras - orações subordinadas com sujeito oculto e se (filtro a): 154

obras - orações principais com sujeito oculto e se (filtro b): 229
obras - orações subordinadas com sujeito oculto e se (filtro b): 1609

obras - orações principais com sujeito oculto e se (filtro c): 4915
obras - orações subordinadas com sujeito oculto e se (filtro c): 6447

bosque - orações principais com sujeito oculto e se (filtro a): 31
bosque - orações subordinadas com sujeito oculto e se (filtro a): 5

bosque - orações principais com sujeito o

24444: Em dezembro de 1932 , a o se *iniciarem* os preparativos para as eleições a a Assembléia Nacional Constituinte , Carvalho Filho participou de a fundação de a Liga de Ação Social e Política ( LASP ) de a Bahia , também chamada de Liga de os Amigos de São Paulo , em uma referência a o movimento constitucionalista paulista de 1932 .
45240: Terminados os trabalhos de a comissão e após ter recusado convite de o interventor federal em Pernambuco , Carlos de Lima Cavalcanti , para *candidatar* se a a Assembléia Nacional
45542: Em 1973 , devido a o afastamento de Quércia de a prefeitura para *candidatar* se a o
46831: A o se *aproximarem* as eleições de outubro de 1950 , considerando que a situação havia mudado -- e que , mais uma vez , Eduardo Gomes não teria condições de se eleger presidente -- Chateaubriand deu um discreto apoio a a candidatura de Vargas .
47152: Segundo declarou a a época a a imprensa , iria se *dedicar* a pavimentar seu caminho para a presidência de a República , a

293: Estudante universitário , *filiou* se a o Partido de o Movimento Democrático Brasileiro ( PMDB ) .
299: Em 1990 , *filiou* se a o Partido Socialista Brasileiro ( PSB ) , por o qual concorreu a as eleições para a Assembleia Legislativa de Pernambuco .
328: *Empenhou* se também em a aprovação de a Lei de Biossegurança , sancionada por o presidente Lula em o dia 24 e março de 2005 , que regulamentou a produção de transgênicos em o país e autorizou a pesquisa científica com células-tronco .
351: *Candidatou* se a a presidência de o Brasil em chapa constituída com Marina Silva , recém-filiada a o PSB , candidata a vice .
359: *Casou* se com Renata de Andrade Lima
370: Com o fim de o Estado Novo ( 1937-1945 ) e a redemocratização de o país , *elegeu* se em o pleito de dezembro de 1945 deputado a a Assembléia Nacional Constituinte por o Pará em a legenda de a União Democrática Nacional ( UDN ) , assumindo o mandato em fevereiro de o ano seguinte .
383: Com a extinção de os partidos polít

8775: *Fazer* se professor ?
9263: E , sem poder *dominar* se : -- Demais , para que precisa você de as notícias de sua tia ?
10073: E foi *assentar* se a o lado de a cortesã , para lhe expor o caso minuciosamente .
10500: Demais , para que *iludir* se ?
11880: Mas para que *iludir* se ?
13201: *Lembrar* se de o subir a estas alturas ! ...
14874: Lobão que , atropeladamente , abriu caminho com três murros , e foi *colocar* se defronte de a criminosa , quando esta ia já ser alcançada por as duas feras .
16361: Vá *aborrecer* se um pouco ...
17396: *Esquecer* se de soprar aquela maldita vela ! ...
17889: Pois se lhe estou a *dizer* que aquelas tenras são tenras de o diabo !
18118: ... *dispor* se a procurar noiva ! ...
18866: E *lembrar* se que ainda havia surras e assassínios irresponsáveis tanto em as fazendas como em as capitais ! ...
18867: *Lembrar* se de que ainda nasciam cativos porque muitos fazendeiros , apalavrados com o vigário de a freguesia batizavam ingênuos como nascidos a

CP299-1: Não possui hoje Vendas Novas praça de toiros , razão por que a corrida aconteceu em uma desmontável , mas António Morais , em o seu livro « A Praça de Toiros de o Campo Pequeno » , a o *referir* se a as outras praças de o país , diz que José Valério construiu uma praça em Vendas Novas em 1862 , tendo em ela havido festas de toiros até 1875 , ano em que soldados de a Escola Prática de Artilharia , por descuido , provocaram o seu incêndio .
CP408-2: Esta é , para David Paiva , uma justificação suficiente para não se *considerar* « directamente responsável » por a situação criada .
CP455-2: Deixou de ser forçoso as pessoas deslocarem se a a sede para se *inscrever* em turnos de 15 dias em um de os centros de férias .
CP522-1: Realce ainda para o mau dia de Marco Pantani , que apenas veio a Espanha para se *preparar* para o Mundial de a Colômbia .
CP530-5: O índice BVL Geral encerrou em baixa , cotando se em os 973,03 pontos , menos 0,23 por cento , enquanto o BPA Contínuo cedeu 0

# Contagem final

In [15]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        if len(tokens):
            i += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if i >= max_examples:
                break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        if len(tokens):
            ii += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if ii >= max_examples:
                break


dhbb - orações principais com sujeito oculto final: 167006 / 480218
dhbb - orações subordinadas com sujeito oculto final: 174986 / 341133

obras - orações principais com sujeito oculto final: 118495 / 353662
obras - orações subordinadas com sujeito oculto final: 141500 / 316297

bosque - orações principais com sujeito oculto final: 1439 / 9364
bosque - orações subordinadas com sujeito oculto final: 2330 / 6842

=== 20 exemplos de sujeito oculto na oração principal em dhbb ===
2: *Estudou* em o Ginásio Diocesano de São Paulo e bacharelou se em 1910 por a Faculdade de Ciências Jurídicas e Sociais .
3: Dedicando se a a advocacia , foi *promotor* público em Cunha ( SP ) e depois delegado de polícia em o Rio de Janeiro , então Distrito Federal .
4: *Iniciou* sua vida política como deputado federal por o Distrito Federal , exercendo o mandato de 1927 a 1929 .
7: Junto com outros companheiros também solidários a o regime deposto e que se haviam asilado em embaixadas e legações , foi *enviado

=== 20 exemplos de sujeito oculto na oração subordinada em obras ===
5: Esclapon e de a duquesa de Mazarin , e esquecia se até de *ouvir* as pilhérias de a magra , feia e adorada Guimard , para só *ter* atenção para o novo escândalo que acabava de surgir inesperadamente .
7: Escreveu logo a o vigário-geral , seu amigo particular , *dando* lhe parte de o fato e pedindo lhe que , sem perda de tempo , *tratasse* de descobrir alguém que o substituísse .
9: Quem teria a coragem de ir , a a última hora , *substituir* La Rose em o púlpito de a capela real , em um de os sermões mais importantes de a Quaresma ? ...
12: E era preciso que ele com efeito estivesse deveras doente , para *faltar* a o sermão de Quinta-feira santa , porque La Rose prezava muito os seus triunfos em a tribuna sacra , e não esperdiçaria facilmente uma boa ocasião de orar perante o rei e toda sua corte de fidalgos , e toda a sua corte de letrados .
13: Entretanto , sabia se também que La Rose , desde que *sentisse* a meno

# Pickle Bosque para posterior reconstituição

In [16]:
import pickle

for corpus in path:
    for sentence in list(root_without_subject[corpus].keys()):
        if not root_without_subject[corpus][sentence]:
            del root_without_subject[corpus][sentence]
        
    for sentence in list(subordinate_without_subject[corpus].keys()):
        if not subordinate_without_subject[corpus][sentence]:
            del subordinate_without_subject[corpus][sentence]

    if corpus == "bosque":
        with open("{}_root.p".format(corpus), "wb") as f:
            pickle.dump(root_without_subject[corpus], f)
        with open("{}_subordinate.p".format(corpus), "wb") as f:
            pickle.dump(subordinate_without_subject[corpus], f)

# Reconstituição

In [1]:
import estrutura_ud
import pickle

with open("bosque_root.p", "rb") as f:
    root = pickle.load(f)
with open("bosque_subordinate.p", "rb") as f:
    subordinate = pickle.load(f)
    
len_root = sum([len(root[x]) for x in root])
len_subordinate = sum([len(subordinate[x]) for x in subordinate])
    
bosque = estrutura_ud.Corpus(recursivo=True)
bosque.load("bosque-ud-2.6.conllu")

dic_pronouns = {
    '1SingMale': ['eu', 'eu'],
    '2SingMale': ['tu', 'tu'],
    '3SingMale': ['ele', 'ele'],
    '3SingFemale': ['ela', 'ele'],
    '1PlurMale': ['nós', 'nós'],
    '2PlurMale': ['vós', 'vós'],
    '3PlurMale': ['eles', 'ele'],
    '3PlurFemale': ['elas', 'ele']
}

bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].deprel = "advcl"
bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].dephead = "13"            

In [2]:
def find_feats(feat, token):
    if not feat in token.feats:
        return ""
    return token.feats.split(feat + "=")[1].split("|")[0]

In [3]:
def insert_phrase(sentence, first_token_id, phrase):
    sintagmas_anteriores = []
    for t, token in enumerate(sentence.tokens):
        if t < sentence.map_token_id[first_token_id]:
            if token.dephead == first_token_id and (token.upos == "ADV" or (token.lemma == "se" and token.upos == "PRON") or token.deprel == "cop" or token.upos == "AUX"):
                is_sintagma = True
                for _token in sentence.tokens:
                    if _token.upos == "PUNCT" and _token.dephead == token.id:
                        is_sintagma = False
                        break
                if is_sintagma:
                    sintagmas_anteriores.append(sentence.tokens[t].id)
    who_head = first_token_id
    first_token_id = min(sintagmas_anteriores) if sintagmas_anteriores else first_token_id

    for new_token in reversed(phrase):
        token = estrutura_ud.Token()
        token.build(new_token.to_str())
        token.dephead = str(int(token.dephead) - int(token.id) + int(who_head.split("-")[0]))
        token.id = first_token_id.split("-")[0]
        if first_token_id == "1":
            if token.word != "SUJ_DESOCULTADO":
                token.word = token.word.title()
            sentence.tokens[0].word = sentence.tokens[0].word.lower()
        token.misc = "|".join(sorted([x for x in token.misc.split("|") + ["SUJ_DESOCULTADO"] if x != "_"]))
        sentence.tokens.insert(sentence.map_token_id[first_token_id], token)
        
        t_first_token_id = sentence.map_token_id[first_token_id]
        for t, token in enumerate(sentence.tokens):
            if t > t_first_token_id:
                token.id = str(int(token.id)+1) if not '-' in token.id else str(int(token.id.split("-")[0])+1) + "-" + str(int(token.id.split("-")[1])+1)
            sentence.map_token_id[token.id] = t
        for t, token in enumerate(sentence.tokens):
            if token.dephead not in ["0", "_"] and sentence.map_token_id[token.dephead] >= sentence.map_token_id[first_token_id]:
                token.dephead = str(int(token.dephead)+1)
        
    sentence.refresh_map_token_id()
                      
    sentence.metadados['text'] = " ".join([x.word for x in sentence.tokens if not '-' in x.id])
    return sentence

In [4]:
#ADVCL à esquerda com mesmo PESSNUM (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del root[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_root))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 13 / 1439

CP40-3:
Quando o povo suíço recusou, em 92, a adesão ao Espaço Económico Europeu, como já fizera com a ONU, cometeu um grave engano.
Quando o povo suíço recusou , em 92 , a adesão a o Espaço Económico Europeu , como já fizera com a ONU , o povo suíço cometeu um grave engano .

CP88-5:
Só quando Moniz Pereira lhe surge na frente, se compenetra de que era mesmo verdade.
Só quando Moniz Pereira lhe surge em a frente , Moniz Pereira se compenetra de que era mesmo verdade .

CP94-5:
Se os chefes militares, actualmente, despacham com o ministro de 15 em 15 dias, farão o mesmo com o primeiro-ministro e o secretário de Estado governa o Ministério.
Se os chefes militares , actualmente , despacham com o ministro de 15 em 15 dias , os chefes militares farão o mesmo com o primeiro-ministro e o secretário de Estado governa o Ministério .

CP493-7:
Como o vírus tem uma propensão para infectar as células humanas, consegue em princípio fazer penetrar o gene dentro das célula

In [5]:
#ADVCL à esquerda com mesmo PESSNUM (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del subordinate[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_subordinate))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 3 / 2330

CP636-2:
Os dirigentes da Fenprof avisam no entanto desde já que se Couto dos Santos insistir nalgumas das directrizes dos seus antecessores arranjará lenha para se queimar.
Os dirigentes de a Fenprof avisam em o entanto desde já que se Couto de os Santos insistir em algumas de as directrizes de os seus antecessores Couto Santos arranjará lenha para se queimar .

CF619-5:
Cabe perguntar: Se o Supremo não está de acordo com a suposta falta de regra das duas moedas, que existem de fato desde a emissão da última medida provisória, por que não a proclamou imediatamente inconstitucional e se serviu dela privadamente?
Cabe perguntar : Se o Supremo não está de acordo com a suposta falta de regra de as duas moedas , que existem de fato desde a emissão de a última medida provisória , por que o Supremo não a proclamou imediatamente inconstitucional e se serviu de ela privadamente ?

CF740-3:
Notamos ainda que, quando um passageiro brasileiro se inscreve num programa de 

In [6]:
#DESINÊNCIA (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for i, t in enumerate(tokens):
        person = find_feats("Person", sentence.tokens[t])
        number = find_feats("Number", sentence.tokens[t])
        gender = find_feats("Gender", sentence.tokens[t])
        if not gender:
            gender = "Male"
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'PRON', '_', feats, "1", 'nsubj', '_', '_']
        else:
            pronoun = ["1", "SUJ_DESOCULTADO", "SUJ_DESOCULTADO", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun))
    
        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), [new_token])
    del root[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_root))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 1426 / 1439

CF896-2:
Dos fumantes, 18,1 milhões são homens e 12,5 milhões, mulheres.
De os fumantes , 18,1 milhões SUJ_DESOCULTADO são homens e 12,5 milhões , mulheres .

CF899-2:
Na sua administração, executou um plano viário com largas avenidas interligando os bairros, que provocaram um boom imobiliário.
Em a sua administração , ele executou um plano viário com largas avenidas interligando os bairros , que provocaram um boom imobiliário .

CF902-3:
Quem não quiser beber durante o jantar, poderá levar o champanhe para casa.
Quem não quiser beber durante o jantar , ele poderá levar o champanhe para casa .

CF904-6:
É o que confirmam as estatísticas do Campeonato Brasileiro, encerrado ontem com o novo título palmeirense.
SUJ_DESOCULTADO é o que confirmam as estatísticas de o Campeonato Brasileiro , encerrado ontem com o novo título palmeirense .

CF908-4:
Diariamente, está promovendo desfiles de moda para seus consumidores.
Diariamente , SUJ_DESOCULTADO está promovendo 

In [7]:
#DESINÊNCIA (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for i, t in enumerate(tokens):
        person = find_feats("Person", sentence.tokens[t])
        number = find_feats("Number", sentence.tokens[t])
        gender = find_feats("Gender", sentence.tokens[t])
        if not gender:
            gender = "Male"
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'PRON', '_', feats, "1", 'nsubj', '_', '_']
        else:
            pronoun = ["1", "SUJ_DESOCULTADO", "SUJ_DESOCULTADO", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun)) 

        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], bosque.sentences[sent_id].tokens[t+i].id, [new_token])
    del subordinate[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_subordinate))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 1934 / 2330

CF893-4:
Para construir o ambiente futurista do filme, Godard recorreu quase que totalmente à fotografia.
Para SUJ_DESOCULTADO construir o ambiente futurista de o filme , Godard recorreu quase que totalmente a a fotografia .

CF895-2:
O problema é político porque envolve, por exemplo, a gratuidade da educação, da saúde, da previdência mínima.
O problema é político porque ele envolve , por exemplo , a gratuidade de a educação , de a saúde , de a previdência mínima .

CF895-3:
Excetuada a educação básica de primeiro grau, que deve ser gratuita e obrigatória, para todo o resto é preciso discriminar quem pode pagar.
SUJ_DESOCULTADO excetuada a educação básica de primeiro grau , que deve ser gratuita e obrigatória , para todo o resto é preciso discriminar quem pode pagar .

CF897-1:
PF prende dois por cobrar ágio
PF prende dois por SUJ_DESOCULTADO cobrar ágio

CF897-3:
A PF (Polícia Federal) indiciou ontem duas pessoas por vender carro com ágio.
A PF ( Polícia F

In [8]:
# Salvar modificado
bosque.save("bosque-ud-2.6-desocultado.conllu")

# Debugagem

In [ ]:
print(corpora['obras'].sentences["540"].to_str())

In [ ]:
#root_nominals["obras"]
root_without_subject["obras"]